In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers datasets sentencepiece

In [30]:
!pip install datasets rouge_score sacrebleu evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=fad2663bfac4d12400a7209ec929d9df871f542a20221db01fc783a35a13968a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 

In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration

In [5]:
dataset_path = '/kaggle/input/recipenlg/RecipeNLG_dataset.csv'
df = pd.read_csv(dataset_path)
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


# Data Sampling

In [6]:
# Randomly sample 10,000 rows from the train dataset
sampled_data = df.sample(n=50000, random_state=42)

# Save the sampled dataset
sampled_data.to_csv("sampled_train.csv", index=False)


In [7]:
#compare stringified lists to python lists
import ast
sampled_data['ingredients'] = sampled_data['ingredients'].apply(ast.literal_eval)
sampled_data['directions'] = sampled_data['directions'].apply(ast.literal_eval)

In [8]:
#create input output pairs
sampled_data['input'] = sampled_data['ingredients'].apply(lambda x: "Ingredients: " + ", ".join(x))
sampled_data['output'] = sampled_data['directions'].apply(lambda x: " ".join(x))

In [9]:
from datasets import Dataset
full_dataset = Dataset.from_pandas(sampled_data)

In [10]:
train_val_df, test_df = train_test_split(sampled_data,test_size=0.1,random_state=42)

In [11]:
# Convert train and test DataFrames to Hugging Face Dataset
train_val_dataset = Dataset.from_pandas(train_val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization

In [12]:
#tokenizing the dataset
tokenizer = T5Tokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [14]:
def tokenize_batch(batch):
  # input_text = batch['input']
  # output_text = batch['output']
  tokenized_input = tokenizer(batch['input'], padding='max_length',truncation=True, max_length=512)
  tokenized_output = tokenizer(batch['output'], padding='max_length',truncation=True, max_length=512)
  tokenized_input['labels'] = tokenized_output['input_ids']
  return tokenized_input

In [15]:
# train_tokenized = train_dataset.map(tokenize_batch, batched=True)
# test_tokenized = test_dataset.map(tokenize_batch, batched=True)

train_val_tokenized = train_val_dataset.map(tokenize_batch, batched=True)
test_tokenized = test_dataset.map(tokenize_batch, batched=True)

#tokenized_dataset.save_to_disk("tokenized_full_dataset")
train_val_tokenized.save_to_disk("tokenized_train_val_dataset")
test_tokenized.save_to_disk("tokenized_test_dataset")

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/45000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# print(train_dataset.column_names)
# print(test_dataset.column_names)

In [16]:
from datasets import load_from_disk

# Load tokenized dataset
train_val_tokenized = load_from_disk("tokenized_train_val_dataset")
test_tokenized = load_from_disk("tokenized_test_dataset")

# Hyperparameter ranges for fine tuning

In [17]:
# Hyperparameter ranges
learning_rates = [1e-5, 1e-4]
num_epochs_list = [5]
batch_size = 8
l2_regularizations = [0.0, 1e-4]

## Cross Validation for Hyperparameter selection

In [18]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)
results = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Fine tuning the t5 model

In [19]:
def collate_fn(batch):
    # Ensure every element in batch is converted to tensor, but ignore strings (such as tokenized words)
    #print(f"First batch item: {batch[0]}")
    #return {key: torch.tensor([d[key] for d in batch if isinstance(d[key], (list, int))]) for key in batch[0]}
    return {
        'input_ids': torch.tensor([d['input_ids'] for d in batch]),
        'attention_mask': torch.tensor([d['attention_mask'] for d in batch]),
        'labels': torch.tensor([d['labels'] for d in batch]),
    }

In [ ]:
best_val_loss = float('inf')
best_state_dict = None
best_hparams = None 

for lr in learning_rates:
    for num_epochs in num_epochs_list:
        for l2_reg in l2_regularizations:
            fold = 0
            fold_losses = []

            # Perform cross-validation
            for train_index, val_index in kf.split(train_val_tokenized):
                fold += 1
                print(f"\nTraining fold {fold} with lr={lr}, epochs={num_epochs}, l2_reg={l2_reg}")

                train_subset = train_val_tokenized.select(train_index)
                val_subset = train_val_tokenized.select(val_index)

                train_dataloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
                
                val_dataloader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

                model = T5ForConditionalGeneration.from_pretrained("t5-small")
                model.to(device)

                # Optimizer with L2 regularization
                optimizer = torch.optim.AdamW(
                    model.parameters(), 
                    lr=lr, 
                    weight_decay=l2_reg
                )
                
                # Training loop
                for epoch in range(num_epochs):
                    model.train()
                    running_loss = 0.0
                    for batch in train_dataloader:
                        optimizer.zero_grad()
                        batch = {key: val.to(device) for key, val in batch.items()}
                        outputs = model(
                            input_ids=batch['input_ids'],
                            attention_mask=batch['attention_mask'],
                            labels=batch['labels']
                        )
                        loss = outputs.loss
                        loss.backward()
                        optimizer.step()
                        running_loss += loss.item()
                    
                    avg_train_loss = running_loss / len(train_dataloader)
                    print(f"Epoch {epoch+1}/{num_epochs} => Train Loss: {avg_train_loss:.4f}")

                # Validation
                model.eval()
                val_loss = 0.0
                for batch in val_dataloader:
                    with torch.no_grad():
                        batch = {key: val.to(device) for key, val in batch.items()}
                        outputs = model(
                            input_ids=batch['input_ids'],
                            attention_mask=batch['attention_mask'],
                            labels=batch['labels']
                        )
                        val_loss += outputs.loss.item()
                
                avg_val_loss = val_loss / len(val_dataloader)
                print(f"Fold {fold} Validation Loss: {avg_val_loss:.4f}")
                fold_losses.append(avg_val_loss)
            
            # Average validation loss for this hyperparameter set
            mean_val_loss = np.mean(fold_losses)
            print(f"\nMean Validation Loss for lr={lr}, epochs={num_epochs}, l2_reg={l2_reg}: {mean_val_loss:.4f}")
            if mean_val_loss < best_val_loss:
                best_val_loss = mean_val_loss
                best_hparams = (lr, num_epochs, l2_reg)


Training fold 1 with lr=1e-05, epochs=5, l2_reg=0.0


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1/5 => Train Loss: 1.3834
Epoch 2/5 => Train Loss: 0.7885
Epoch 3/5 => Train Loss: 0.7433
Epoch 4/5 => Train Loss: 0.7196
Epoch 5/5 => Train Loss: 0.7032
Fold 1 Validation Loss: 0.6456

Training fold 2 with lr=1e-05, epochs=5, l2_reg=0.0
Epoch 1/5 => Train Loss: 1.3824
Epoch 2/5 => Train Loss: 0.7933
Epoch 3/5 => Train Loss: 0.7496
Epoch 4/5 => Train Loss: 0.7250
Epoch 5/5 => Train Loss: 0.7087
Fold 2 Validation Loss: 0.6382

Training fold 3 with lr=1e-05, epochs=5, l2_reg=0.0
Epoch 1/5 => Train Loss: 1.3861
Epoch 2/5 => Train Loss: 0.7872
Epoch 3/5 => Train Loss: 0.7431
Epoch 4/5 => Train Loss: 0.7199
Epoch 5/5 => Train Loss: 0.7038
Fold 3 Validation Loss: 0.6443

Mean Validation Loss for lr=1e-05, epochs=5, l2_reg=0.0: 0.6427

Training fold 1 with lr=1e-05, epochs=5, l2_reg=0.0001
Epoch 1/5 => Train Loss: 1.4026
Epoch 2/5 => Train Loss: 0.7891
Epoch 3/5 => Train Loss: 0.7435
Epoch 4/5 => Train Loss: 0.7195
Epoch 5/5 => Train Loss: 0.7033
Fold 1 Validation Loss: 0.6461

Training

## Final training with the best set of coefficients

In [21]:
model.save_pretrained("/kaggle/working/fine_tuned_t5")
tokenizer.save_pretrained("/kaggle/working/fine_tuned_t5")

('/kaggle/working/fine_tuned_t5/tokenizer_config.json',
 '/kaggle/working/fine_tuned_t5/special_tokens_map.json',
 '/kaggle/working/fine_tuned_t5/spiece.model',
 '/kaggle/working/fine_tuned_t5/added_tokens.json')

In [20]:
best_hparams = (1e-04, 5, 0.0001)
lr_best, epochs_best, l2_best = best_hparams
final_dl = DataLoader(train_val_tokenized, batch_size=batch_size,
                      shuffle=True, collate_fn=collate_fn)

model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr_best, weight_decay=l2_best)

for epoch in range(epochs_best):
    model.train(); running = 0
    for batch in final_dl:
        optimizer.zero_grad()
        batch = {k:v.to(device) for k,v in batch.items()}
        loss = model(**batch).loss
        loss.backward(); optimizer.step()
        running += loss.item()
    print(f"final‑train epoch {epoch+1}/{epochs_best} loss {running/len(final_dl):.4f}")

# save final model ONCE
model.save_pretrained("/kaggle/working/fine_tuned_t5_2")
tokenizer.save_pretrained("/kaggle/working/fine_tuned_t5_2")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


final‑train epoch 1/5 loss 0.7615
final‑train epoch 2/5 loss 0.6341
final‑train epoch 3/5 loss 0.6006
final‑train epoch 4/5 loss 0.5794
final‑train epoch 5/5 loss 0.5640


('/kaggle/working/fine_tuned_t5_2/tokenizer_config.json',
 '/kaggle/working/fine_tuned_t5_2/special_tokens_map.json',
 '/kaggle/working/fine_tuned_t5_2/spiece.model',
 '/kaggle/working/fine_tuned_t5_2/added_tokens.json')

# Generating Recipes

In [23]:
def generate_recipe(ingredients):
    input_text = f"Ingredients: { ', '.join(ingredients)}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [26]:
ingredients = ['1 dozen eggs', '1 cup flour', '1 gallon milk', '1 cup sugar', '1 oz baking powder']
print(generate_recipe(ingredients))

Preheat oven to 350 degrees F. Grease and flour a 9x13 baking pan. In a large mixing bowl, mix together the eggs, flour, milk, sugar, and baking powder. Stir in the egg mixture and mix well. Pour into prepared pan. Bake for 20 minutes or until a toothpick comes out clean.


# Evaluating the model

In [28]:
#from tdqm import tdqm
def model_evaluate(test_dataloader,model,device,tokenizer):
    model.eval()
    predictions=[]
    references=[]
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        reference_texts = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)
        with torch.no_grad():
            outputs = model.generate(input_ids,attention_mask=attention_mask, max_length=512, num_beams=4, early_stopping=True)
        generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(generated_texts)
        references.extend(reference_texts)
    return predictions,references

In [31]:
import evaluate
bleu_metric = evaluate.load('sacrebleu')
rouge_metric = evaluate.load('rouge')

In [32]:
def compute_metrics(predictions,references):
    bleu_score = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
    rouge_result = rouge_metric.compute(predictions=predictions, references=references)
    
    return {"bleu": bleu_score['score'], "rouge": rouge_result}

In [33]:
test_dataloader = DataLoader(test_tokenized, batch_size=batch_size, collate_fn=collate_fn)
predictions,references = model_evaluate(test_dataloader,model,device,tokenizer)
metrics = compute_metrics(predictions, references)
print(f"BLEU Score: {metrics['bleu']}")
print(f"ROUGE Scores: {metrics['rouge']}")

BLEU Score: 7.249748550179903
ROUGE Scores: {'rouge1': 0.33777885202650104, 'rouge2': 0.11686751416760655, 'rougeL': 0.2467904676888479, 'rougeLsum': 0.24689741359084372}


# Downloading the model

In [21]:
import shutil
from IPython.display import FileLink
shutil.make_archive('/kaggle/working/fine_tuned_t5_2','zip','/kaggle/working/fine_tuned_t5_2')
from IPython.display import FileLink

# Create a download link for the zipped model
FileLink("/kaggle/working/fine_tuned_t5_2.zip")


/kaggle/working/fine_tuned_t5_2.zip